In [ ]:
import torch
import numpy as np
import face_recognition as fc
import joblib
import cv2
from datetime import datetime, timedelta

In [ ]:
with open('face_model.pkl', 'rb') as f:
    face_model = joblib.load(f)
# Model
main_model = torch.hub.load('ultralytics/yolov5', 'custom', path='main_model.pt')
face_cascade = cv2.CascadeClassifier(cv2.samples.findFile(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'))

In [ ]:
def facerecog(vid):
  rgb=cv2.cvtColor(vid,cv2.COLOR_BGR2RGB)
  Loc=fc.face_locations(rgb)
  encodings = fc.face_encodings(rgb,Loc)
  for encoding in encodings:
    result=fc.compare_faces(face_model["encodings"],encoding)
    face_dis=fc.face_distance(face_model["encodings"],encoding)
    prob=min(face_dis)
    if prob>0.50:
      name="Unknown"
    else:
      if True in result:
        matchedIdxs = [i for (i, b) in enumerate(result) if b]
        counts = {}
        for i in matchedIdxs:
          name = face_model["names"][i]
          counts[name] = counts.get(name, 0) + 1
          name = max(counts, key=counts.get)
    return name

In [ ]:
lst = []
def detectedPerson(name):
    expiration_time = datetime.now() + timedelta(seconds=15)
    return dict({"name":name,"exp":expiration_time})

In [ ]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    for values in lst:
        if values["exp"]<datetime.now():
            lst.remove(values)
    
    # Make detections 
    results = main_model(img)
    # Results
  df = results.pandas().xyxy[0]
  list1=[]    
  for i in df['name']: # name->labels
    list1.append(i)  # or .show(), .save(), .crop(), .pandas(), etc.
  if ("helmet" in lst) and ("glasses" in lst) and ("headphones" in lst):
    continue
  else:
    final=facerecog(img)
    if final==None:
      continue
    else:
      print(final)
      duplicate = False
      for i in range(len(lst)):
        if lst[i]["name"] ==final:
          duplicate = True
      if(duplicate):
        print("name already added")
      else:
        lst.append(detectedPerson(final))
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
cap.release()
cv2.destroyAllWindows()